### <b> 단기 투자 시 수익 성과가 좋은 팩터 전략 <b>
    1. 밸류 팩터(1m) : 저pbr, 저per (단, per>0이고 pbr>0인 종목)
    2. 어닝 모멘텀(1m) : EPS 변화율이 높은 종목
    3. 가격 모멘텀
    - 이격도 리버전 : 단기 투자 전략에 주가/직전 1개월 평균주가 하위 종목을 선택합니다 
    - 주가 리버전 (1m) : 직전 1개월 주가 수익률 하위 종목 투자
    4. 트레이딩 팩터 : 개인 순매수 (1m)

In [359]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import json
import requests

from pykrx import stock

### 1.가치주 : 저per, 저pbr

In [427]:
data = stock.get_market_cap("20240321", market="ALL") #3개월의 투자 수익률 확인 위해
data = data[['종가', '시가총액']]
data.head()

,종가,시가총액
티커,,
005930,79300,473403756215000
000660,170000,123760402050000
373220,413500,96759000000000
207940,857000,60996118000000
005935,65800,54145944860000


In [428]:
#api 활용 데이터 - per, pbr
df = stock.get_market_fundamental_by_ticker("20240321", market='ALL')
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,100.17,3.17,30,0.00,0
095570,8076,24.35,0.61,201,5.52,270
006840,41948,0.00,0.37,0,1.30,200
054620,10864,14.77,0.69,505,0.00,0
265520,17980,3.83,1.17,5463,1.29,270


In [429]:
df.describe() #per<5 고정 #PBR의 경우 상위 25% 값보다 작은 것

,BPS,PER,PBR,EPS,DIV,DPS
count,2.623000e+03,2623.000000,2623.000000,2623.000000,2623.000000,2623.000000
mean,2.195296e+04,32.057278,2.356824,1718.106748,1.267118,343.637819
std,1.204459e+05,271.506101,11.063590,10136.408705,2.120688,1160.318840
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.453500e+03,0.000000,0.490000,0.000000,0.000000,0.000000
50%,4.735000e+03,5.770000,0.970000,143.000000,0.000000,0.000000
75%,1.257750e+04,17.400000,2.060000,938.000000,1.935000,200.000000
max,4.906107e+06,8700.000000,490.290000,406090.000000,35.250000,20000.000000


In [430]:
#업데이트 편의를 위해 넘파이 사용
import numpy as np
column_pbr = df['PBR'].values
pbr_25 = np.percentile(column_pbr, 25)

df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < pbr_25)] #저per, 저pbr인 데이터
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
006840,0.00,0.37
138930,3.48,0.27
001465,0.00,0.00
035760,0.00,0.44
00104K,0.00,0.00
...,...,...
028080,0.00,0.45
079980,0.00,0.34
005010,0.90,0.32


In [431]:
df2.index

Index(['006840', '138930', '001465', '035760', '00104K', '011155', '001045',
       '097955', '000480', '456440',
       ...
       '002460', '000850', '032560', '192410', '115160', '028080', '079980',
       '005010', '000540', '000545'],
      dtype='object', name='티커', length=496)

In [432]:
df3 = pd.merge(data, df2, on='티커')
df3

,종가,시가총액,PER,PBR
티커,,,,
005935,65800,54145944860000,0.00,0.00
015760,22850,14668879159450,0.00,0.36
024110,15160,12088976174040,4.57,0.42
316140,15290,11497307258690,3.65,0.39
011200,15890,10948837591440,0.77,0.38
...,...,...,...,...
001525,5850,3613726350,0.00,0.00
002785,4200,3600315600,0.00,0.00
002995,12030,3515959980,0.00,0.00


### 1.2 PER > 0, PBR > 0

In [433]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3 = df3[['종가', 'PER', 'PBR']]
df3.shape 

(142, 3)

In [493]:
df3.index

Index(['024110', '316140', '011200', '078930', '004020', '088350', '138930',
       '036460', '000880', '139480',
       ...
       '000950', '012620', '050120', '225590', '054940', '377460', '073540',
       '001770', '106240', '115570'],
      dtype='object', name='티커', length=142)

   ### 3. EPS 변화율 
   - 1m 기준

In [436]:
#eps변화율 분석(1m)
df = stock.get_market_fundamental("20240203", "20240304", '006840', freq="m")
df = df.T
df

날짜,2024-02-29,2024-03-31
BPS,41948.00,41948.00
PER,0.00,0.00
PBR,0.41,0.39
EPS,0.00,0.00
DIV,1.16,1.24
DPS,200.00,200.00


In [437]:
import pandas as pd

ticker = df3.index
eps_change_data = []

for stock_code in ticker:
    df = stock.get_market_fundamental("20240203", "20240304", stock_code, freq="m")
    df = df.T
    df['변화율'] = (df.iloc[:, 1] - df.iloc[:, 0]) / df.iloc[:, 0]
    eps_change_data.append(df['변화율'])

# DataFrame 생성
eps_change_df = pd.DataFrame(eps_change_data, index=ticker)
eps_change_df

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
024110,0.0,0.029197,0.026316,0.0,-0.028409,0.0
316140,0.0,-0.008523,0.000000,0.0,0.007833,0.0
011200,0.0,-0.064516,-0.066667,0.0,0.068471,0.0
078930,0.0,-0.009132,0.000000,0.0,0.009901,0.0
004020,0.0,-0.008772,0.000000,0.0,0.007042,0.0
...,...,...,...,...,...,...
377460,0.0,0.000000,0.000000,0.0,NaN,NaN
073540,0.0,-0.011905,0.000000,0.0,NaN,NaN
001770,0.0,-0.058824,-0.060000,0.0,0.063830,0.0


In [438]:
eps_change_df['EPS'].mean()

0.0

In [501]:
eps_change_df = eps_change_df.loc[eps_change_df['EPS'] <= 0]
ticker = eps_change_df.index.to_list()
ticker

['024110',
 '316140',
 '011200',
 '078930',
 '004020',
 '088350',
 '138930',
 '036460',
 '000880',
 '139480',
 '004170',
 '047040',
 '139130',
 '137310',
 '375500',
 '006360',
 '001120',
 '023590',
 '000670',
 '006120',
 '003380',
 '003240',
 '016380',
 '007700',
 '005880',
 '383800',
 '032190',
 '017940',
 '377740',
 '020000',
 '058650',
 '123890',
 '001940',
 '093050',
 '104700',
 '003300',
 '306200',
 '151860',
 '007690',
 '005090',
 '084010',
 '030530',
 '001500',
 '003120',
 '035890',
 '029460',
 '000480',
 '003800',
 '000540',
 '005010',
 '009160',
 '007860',
 '001230',
 '016590',
 '013120',
 '213500',
 '035080',
 '037710',
 '004360',
 '002020',
 '030210',
 '003200',
 '035510',
 '038540',
 '023760',
 '007160',
 '044450',
 '092790',
 '009580',
 '023600',
 '003070',
 '014280',
 '363280',
 '121440',
 '317400',
 '000320',
 '039560',
 '003830',
 '104480',
 '000700',
 '012700',
 '037460',
 '024090',
 '004150',
 '005710',
 '013580',
 '053700',
 '009070',
 '032560',
 '005960',
 '052330',

### 4. 주가 리버전 (1m)
- 직전 1 달간 주가수익률 하위 투자 . 주가 모멘텀 (1m) 팩터의 반대

In [560]:
df = stock.get_market_ohlcv("20240220", "20240320", '024110', "m") #직전 한달 주가 수익률
gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
gain

5.857142857142858

In [561]:
df

,시가,고가,저가,종가,거래량
날짜,,,,,
2024-02-29,14140,14160,12920,14000,15093218
2024-03-31,13880,16010,13770,14820,26581904


In [580]:
import pandas as pd

tickers = eps_change_df.index.to_list()
gains = []

for ticker in tickers:
    df = stock.get_market_ohlcv("20240220", "20240320", ticker, "m")
    gain = ((df.iloc[1,3] - df.iloc[0,3]) / df.iloc[0,3]) * 100
    gains.append(gain)

result_df = pd.DataFrame({'ticker': tickers, 'gain': gains})
result_df.set_index('ticker', inplace=True)

In [581]:
result_df.describe()

,gain
count,142.000000
mean,-2.170007
std,5.795440
min,-14.066776
25%,-5.923904
50%,-3.072443
75%,0.867691
max,28.631285


In [595]:
gain_25 = result_df['gain'].quantile(0.25)
selected_rows = result_df['gain'] < gain_25

temp_df = result_df[selected_rows]
temp_df.index.nunique() #36개의 종목 필터링

36

In [598]:
temp_df.index

Index(['011200', '004020', '139480', '137310', '000670', '003380', '003240',
       '016380', '005880', '032190', '151860', '029460', '000540', '013120',
       '004360', '044450', '003070', '014280', '363280', '317400', '104480',
       '012700', '013580', '053700', '002460', '241770', '024800', '091590',
       '016250', '004870', '050860', '290120', '050120', '073540', '106240',
       '115570'],
      dtype='object', name='ticker')

### 5. 이격도 리버전 (1m)
- 저이격도 선택

In [614]:
data = stock.get_market_ohlcv("20240220", "20240320", "011200", "d")
mean = data['종가'].mean()
close_temp = stock.get_market_ohlcv("20240320","20240320", "011200", "d")
close = close_temp.iloc[0,3]
revision = (close / mean) * 100
revision

90.05736137667306

In [616]:
tickers = temp_df.index.tolist()
revisions = []

for ticker in tickers:
    data = stock.get_market_ohlcv("20240220", "20240320", ticker, "d")
    mean = data['종가'].mean()
    close_temp = stock.get_market_ohlcv("20240320", "20240320", ticker, "d")
    close = close_temp.iloc[0,3]
    revision = (close / mean) * 100
    revisions.append(revision)

# 새로운 데이터프레임 생성
result_df = pd.DataFrame({'ticker': tickers, 'revision': revisions})
result_df.set_index('ticker', inplace=True)

In [618]:
result_df.describe()

,revision
count,36.000000
mean,94.555307
std,1.507134
min,90.057361
25%,93.746465
50%,94.881515
75%,95.517984
max,97.485655


In [622]:
revision_50 = result_df['revision'].quantile(0.5)
selected_rows = result_df['revision'] < revision_50

revision_df = result_df[selected_rows]
revision_df.index.nunique() 

18

In [623]:
revision_df.index

Index(['011200', '000670', '003240', '005880', '000540', '013120', '044450',
       '363280', '317400', '104480', '053700', '091590', '004870', '050860',
       '290120', '050120', '073540', '115570'],
      dtype='object', name='ticker')

### 6. 개인 순매수 (1m)
- 직전 한달 간 개인 순매수대금 / 시가총액 *100
- 고개인매수 종목

In [512]:
data1 = stock.get_market_trading_value_by_date("20240220", "20240321", '024110')[['기관합계','개인']] #직전 한달간 개인 순매수 대금
data2 = stock.get_market_cap("20240220", "20240321", '024110')['시가총액']
data1['개인순매수'] = (data1['개인'] / data2) * 100
data1 = data1[['개인', '개인순매수']]
data1['개인순매수'].mean()

-0.005353287718044882

In [625]:
tickers = ['011200', '000670', '003240', '005880', '000540', '013120', '044450',
       '363280', '317400', '104480', '053700', '091590', '004870', '050860',
       '290120', '050120', '073540', '115570']
mean_values = []

for ticker in tickers:
    data1 = stock.get_market_trading_value_by_date("20240220", "20240321", ticker)
    data2 = stock.get_market_cap("20240220", "20240321", ticker)['시가총액']
    data1['개인순매수'] = (data1['개인'] / data2) * 100
    mean = data1['개인순매수'].mean()
    mean_values.append(mean)

final_df = pd.DataFrame({'ticker': tickers, 'mean': mean_values})
final_df.set_index('ticker', inplace=True)
final_df

,mean
ticker,
011200,0.018293
000670,0.014361
003240,-0.001747
005880,0.041811
000540,-0.010833
013120,0.022019
044450,0.020827
363280,0.044014
317400,0.015730


In [626]:
final_df.describe()

,mean
count,18.000000
mean,0.014582
std,0.014697
min,-0.010833
25%,0.000946
50%,0.017386
75%,0.020481
max,0.044014


In [630]:
final_50 = final_df['mean'].quantile(0.5)
selected_rows = final_df['mean'] < final_50

screening = result_df[selected_rows]
screening.index.nunique() 

9